# コンピュータと音楽：松原担当分第3回

## 今日の目標
- Pythonの環境構築
- GitHubの環境構築
- MIDIファイルの読み込み
- MIDIファイルの再生
- MIDIファイルの編集と書き込み
- MIDIファイルから楽譜情報を書き出す（本日の課題）

## Pythonの環境構築
- Anaconda 3.6をインストールする
- ターミナルでmidoモジュールとRtMidiモジュールのインストール

```bash
% sudo pip install mido
% sudo pip install python-rtmidi
```


## GitHubの環境構築
- GitHub Desktop(https://desktop.github.com)のインストール
- GitHubのアカウント取得(履修生はこっちは必要ないかも？アカウントなしでGitHubDesktopが起動できるかどうか？)
 - 学生は非公開に設定できる

## 準備
- SimpleSynth(http://notahat.com/simplesynth/)やFluidSynthなどのMIDI Synthesizerをインストール
- MIDIデータのダウンロード（manabaより）
 - データ元はStanford大のCCARH (http://www.ccarh.org) 
- 授業用レポジトリをGitHubDesktopを使って手元の任意の場所にクローン
 - https://github.com/masaki-cb/MediaExpA2017
  - 若狭くん向け。学生にもやらせる？＞自分のGitHubにforkしてから、GitHubDesktopにcloneしたのちbranch offして手元のjupyterなどで更新、更新後、GitHubDesktopでcommit onすると枝上の編集過程が保存される。私のリポジトリにpull request出して私が承認すると若狭くんの修正が反映される。そのあとsyncなどでmasterが変更されているのを確認する。http://qiita.com/yukiyan/items/2ea3dc5813fdba5d9cd2
- JupyterNotebookを起動しipynbファイル（このファイル）を読み込み

### 準備その2（pythonの練習）
- 何か適当なチュートリアルipynbを探す
 - 記述スタイル（インデント）
 - 主な型（int, float, string, list, sets, dict）
 - 演算子
 - 浅いコピーと深いコピー
 - 制御文（if, forなど）
 - 関数
 - ライブラリ（モジュールとパッケージ）

あたりが理解できると良い

## MIDIファイルの読み込み
### midoモジュールをインポート
詳しいドキュメントは https://mido.readthedocs.io/en/latest/ を参照

In [1]:
import mido

### MIDIファイルを開く
mido.MidiFile('__MIDI ファイル名__')  

In [2]:
mid = mido.MidiFile('./data/invention/BWV772.mid')

エラーがなければ読み込みが正常に行われる。何がともあれprint( )で確認。

In [3]:
print(mid)

<midi file './data/invention/BWV772.mid' type 1, 3 tracks, 949 messages>


読み込まれた mid は MidiFile 型であることがわかる。"mid." を打ち込んでタブを押すとタブ補完機能により、メソッドや変数へのアクセス候補が列挙される。もっと詳しく調べる場合は doc を読んだり midifile.py を見ること。

In [4]:
print(mid.tracks)

[<midi track '' 3 messages>, <midi track '' 506 messages>, <midi track '' 440 messages>]


### MIDI ファイルの内容を出力
`mido.MidiFile.print_tracks()` でMIDIファイルの内容を出力できる。

In [5]:
mid.print_tracks()

=== Track 0
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message set_tempo tempo=697674 time=0>
<meta message end_of_track time=0>
=== Track 1
<meta message text text='RH' time=0>
<message note_on channel=0 note=60 velocity=64 time=60>
<message note_off channel=0 note=60 velocity=64 time=60>
<message note_on channel=0 note=62 velocity=64 time=0>
<message note_off channel=0 note=62 velocity=64 time=60>
<message note_on channel=0 note=64 velocity=64 time=0>
<message note_off channel=0 note=64 velocity=64 time=60>
<message note_on channel=0 note=65 velocity=64 time=0>
<message note_off channel=0 note=65 velocity=64 time=60>
<message note_on channel=0 note=62 velocity=64 time=0>
<message note_off channel=0 note=62 velocity=64 time=60>
<message note_on channel=0 note=64 velocity=64 time=0>
<message note_off channel=0 note=64 velocity=64 time=60>
<message note_on channel=0 note=60 velocity=64 time=0>
<message note_off 

## MIDIファイルの再生

### MIDI シンセの起動とoutputデバイスの検索
SimpleSynth を起動し、`mido.get_output_names()` でMIDI outputデバイスの確認しよう。

In [6]:
mido.get_output_names()

['SimpleSynth virtual input']

### MIDIファイルの再生
`mido.MifiFile.play()` で MIDI メッセージを delta time 経過させながら出力する。出力先が MIDI synthesizer であれば `mido.ports.BaseOutput.send(`_msg_`)` で音がなる。停止方法はInterrupt kernelボタンを押す（他に良い方法を見つけたら教えてください）。

In [7]:
with mido.open_output('SimpleSynth virtual input') as port:
    # 引数が空の場合はデフォルト選択。withにより終了時にポートを閉じる
    for msg in mid.play():
        port.send(msg)

## MIDIの編集と作成
### MIDI メッセージへのアクセス方法
mido.MidiFile型のtracks変数はmido.MidiTrack型のlist構造である。
またmido.MidiTrack型はlist型のサブクラスであり、中身はmido.Message型のlist構造である。
従って二重配列のように任意のトラックの任意のMIDIメッセージへアクセス可能である。

In [8]:
print(mid.tracks[1][1])

note_on channel=0 note=60 velocity=64 time=60


type, channel, velocity, timeでそれぞれの情報にアクセスできる。

In [9]:
print(mid.tracks[1][1].note)

60


### MIDIの編集
移調の例：全てのノート番号を全音分下げてみよう。

In [10]:
for track in mid.tracks:
    for msg in track:
        if msg.type in ['note_on', 'note_off']:
            msg.note -= 2

人間らしさの付与の例：音符の強さと長さに揺らぎをいれてみよう。

In [11]:
import random
for track in mid.tracks:
    for msg in track:
        if msg.type == 'note_on' and msg.velocity > 40:
            msg.velocity += random.randint(-5, 5)
        if msg.type == 'note_off' and msg.time > 30:
            msg.time += random.randint(-2, 2)

### MIDIの作成
`mido.MidiFile.save(`__ファイル名__`)`でMIDIファイルを書き出す。

In [12]:
mid.save('hoge.mid')

midoモジュールがインストールされていればターミナル上で`mido-play`で再生できる。先ほど書き出したMIDIファイルを再生して確認してみよう。

```bash
% mido-play MIDIファイル
```

## MIDIファイルから楽譜情報を書き出す（本日の課題）
`print_tracks()`ではトラックごとのMIDI メッセージの列挙のみであった。音楽的な分析や編集をするためには楽譜情報に変換する必要がある。

MIDIファイルを読み込み、下記のような楽譜情報（音名、オクターブ、ヴェロシティ、音長、チャンネル、発音時刻）を書き出すプログラムを作成せよ。ただし四分音符の長さを1とする。

（提出方法をどうするか？？ipynbをhtmlなどに書き出してmanabaにて提出？）

```
Note:  Vel:   Dur : ch :  time
----:-----:-------:----:-------
C 6 :  84 : 0.250 :  0 : 1.250
B 5 :  84 : 0.250 :  0 : 1.500
A 5 :  84 : 0.250 :  0 : 1.750
G 5 :  84 : 0.250 :  0 : 2.000
```

ヒント：
- 各メッセージの時刻が必要
- 音長＝同じノートナンバーのノートオンとノートオフの時刻の差
- トラックごとに計算したあとは時刻やソートを使う(下記参照)

In [14]:
from operator import attrgetter
sorted(mid.tracks[1][1:-1], key=attrgetter('note', 'time'))
# note, timeの順でソート

[<message note_on channel=0 note=58 velocity=64 time=0>,
 <message note_off channel=0 note=58 velocity=64 time=59>,
 <message note_on channel=0 note=58 velocity=62 time=60>,
 <message note_off channel=0 note=58 velocity=64 time=60>,
 <message note_on channel=0 note=60 velocity=63 time=0>,
 <message note_on channel=0 note=60 velocity=63 time=0>,
 <message note_on channel=0 note=60 velocity=60 time=0>,
 <message note_on channel=0 note=60 velocity=66 time=0>,
 <message note_off channel=0 note=60 velocity=64 time=58>,
 <message note_off channel=0 note=60 velocity=64 time=59>,
 <message note_off channel=0 note=60 velocity=64 time=59>,
 <message note_off channel=0 note=60 velocity=64 time=120>,
 <message note_on channel=0 note=62 velocity=66 time=0>,
 <message note_on channel=0 note=62 velocity=64 time=0>,
 <message note_on channel=0 note=62 velocity=63 time=0>,
 <message note_on channel=0 note=62 velocity=69 time=0>,
 <message note_on channel=0 note=62 velocity=61 time=0>,
 <message note_on